In [5]:
print("ok")

ok


In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'd:\\Projects AI\\End-to-End-Medical-Chatbot-Generative-AI'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [12]:
# Extract Data From the PDF File
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [16]:
extracted_data=load_pdf_file(data='Data/')

In [18]:
# extracted_data